## Project Goals
- Find drivers for customer churn at Telco. Why are customers churning?

- Construct a ML classification model that accurately predicts customer churn

- Present your process and findings to the lead data scientist

## Acquire

- I have 7043 rows and 24columns
- 6 of the 24 columns are numric while 18 of them are string object colums
- out of the 6 numeric columns 1 is a float colum
- I have no null values in my entire data
- I also see the descriptive statistics of my data

## Prepare

**Removed columns:**

- `custromer_id` doesn't look important in predicting churn, so I will remove it from my dataframe

- `internet_service_type_id` and `internet_service_type` are relaying the same information, so I will remove internet_service_type_id beause it is valuable for me to know the name of the type of internet.

- `contract_type` and `contract_type_id` are also relaying the same information, so I will remove contract_type_id because it is important that I know the name of the contract.

- `payment_type` and `payment_type_id` are also relaying the same information, so I will remove payment_type_id because it is important that I know the name of the type of payment.


**Check data types:**

- total_charges` column is a string buy that is not the correect data type, so I will covert it into float.

    - This column contains 11 rows filled by empty spaces (" ") into them, so I will replace them with `mean` of the the column.
    
**What I see on numeric columns:**

- `senior_citizen` contains 2 binary numbers and value 0 contains 84% all all the data. This means that only 16% are sinior sitizens.
- `tenure` is a continious variable and it is the number of month a customer has been witht the telco's services.
- `monthly_charges`: is continious variable and it is the amount each customer is being charged every month.
- `total_charges` is a continious variable and it is the sum of all the payments completed by the customer.

**What I see on object columns:**

- `gender` is a two choice categorical variable and the counts are split about equaly on both directions.
- `partner` is a two choice categorical variable and the counts are split about equaly on both directions. It identifies the customers who have have a spouse on the line.
- `dependents` is a two choice categorical variable and about 70% of them don't have dependents.
- `multiple_lines` is a two choice categorical variable and only a small count (about 6% more) of customers more have don't have multiple lines.
- `online_security` is a three choice categorical variable and about 50% of them don't have online security while they have internet service.
- `online_backup` is a three choice categorical variable and about 43% of them don't have online backup while they haveinternet service.
- `device_protection` is a three choice categorical variable and about 43% of them don't have device protection while they have internet service.
- `tech_support` is a three choice categorical variable and about 50% of them don't receive technical support while they have internet service.
- `stream_tv` is a three choice categorical variable and about 40% of them don't rdon't stream tv while they have internet service.
- `stream_movies` is a three choice categorical variable and about 40% of them don't rdon't stream movies while they have internet service.
- `paperless_billing` is a two choice categorical variable and about 60% of them have paperless billing
- `churn` is a two choice categorical variable and about 20% of the customers are churn.
- `contract_type` is a three choice categorical variable and about 55% of the customers are month to month.
- `internet_service_type` is a three choice categorical variable and about 44% of the customers have fiber optic.
- `payment_type` is a three choice categorical variable and about 33% of the customers use electronice check.



**More about prepare in big file (Must get later)**